## Task 1

In [1]:
def convert_line_algo_1(string: str) -> tuple:
    weight, length = list(map(int, string[:-1].split(' ')))
    return (weight - length, weight, length)

with open('jobs.txt') as f:
    weight_list_algo_1 = list(sorted([convert_line_algo_1(item) for item in f.readlines()[1:]], reverse=True))
    
def calc_sum(weight_list: list) -> int:
    W = [item[1] for item in weight_list]
    C = [weight_list[0][2]]
    for item in weight_list[1:]:
        C.append(item[2] + C[-1])
    print(C[:10])
    return sum([W[i] * C[i] for i in range(len(C))])

calc_sum(weight_list_algo_1)

[1, 4, 7, 9, 11, 12, 14, 16, 18, 22]


69119377652

## Task 2

In [2]:
def convert_line_algo_2(string: str) -> tuple:
    weight, length = list(map(int, string[:-1].split(' ')))
    return (weight/length, weight, length)

with open('jobs.txt') as f:
    weight_list_algo_2 = list(sorted([convert_line_algo_2(item) for item in f.readlines()[1:]], reverse=True))
    
calc_sum(weight_list_algo_2)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


67311454237

# Prim's Algorithm

## Min Heap With Mapping

1. all elemends added to heap are tuples (key, node)
2. all nodes have to be unique
3. heap remember position of each node

In [3]:
from typing import Tuple, Optional

class MinHeapMap:
    def __init__(self):
        self.h = [] 
        self.map = {}
        
    def insert(self, item: Tuple[int]):
        self.h.append(item)
        key, node = item # get node
        self.map[node] = len(self) - 1 # remember node position
        self.bubble_up(len(self) - 1)
        
    def bubble_up(self, idx: int):
        parent = self.parent(idx)
        while parent is not None:
            if self.h[idx] < self.h[parent]:
                idx_node, parent_node = self.h[idx][1], self.h[parent][1] # get nodes
                self.map[idx_node], self.map[parent_node] = parent, idx # swap nodes positions
                self.h[idx], self.h[parent] = self.h[parent], self.h[idx]
                idx, parent = parent, self.parent(parent)
            else:
                break
    
    def get_min(self) -> Optional[Tuple[int]]:
        if self.h:
            return self.h[0]
    
    def pop_min(self) -> Tuple[int]: 
        min_item = self.h[0]
        _, min_node = min_item # get min_node
        self.map.pop(min_node) # remove min_node from the mapping
        last_item = self.h.pop()
        if len(self) > 0:
            self.h[0] = last_item
            _, last_node = last_item # get last_node
            self.map[last_node] = 0 # set last_node map position as 0
            self.bubble_down(0)
        return min_item

    def bubble_down(self, idx: int): 
        l_child, r_child = self.childs(idx)
        while l_child < len(self):
            if r_child >= len(self) or self.h[l_child] < self.h[r_child]:
                min_child = l_child
            else:
                min_child = r_child
                
            if self.h[idx] > self.h[min_child]:
                idx_node, min_child_node = self.h[idx][1], self.h[min_child][1] # get nodes
                self.map[idx_node], self.map[min_child_node] = min_child, idx # swap map positions
                self.h[idx], self.h[min_child] = self.h[min_child], self.h[idx]
                idx, (l_child, r_child) = min_child, self.childs(min_child)
            else:
                break
    
    def parent(self, idx: int) -> Optional[int]:
        if idx > 0:
            return (idx - 1) >> 1
        
    def childs(self, idx: int) -> Tuple[int]:
        l_child = idx << 1 
        return l_child + 1, l_child + 2
    
    def __len__(self) -> int:
        return len(self.h)
    
    def pop_node(self, node: int) -> Tuple[int]: # now we can delete any node from heap for O(h) time
        node_idx = self.map.pop(node)
        node_key, _ = self.h[node_idx]
        last_key, last_node = self.h.pop()
        if node_idx != len(self):
            self.h[node_idx] = (last_key, last_node)
            self.map[last_node] = node_idx
            self.bubble_down(node_idx)
            self.bubble_up(node_idx)
        return node_key, node
    
    def __contains__(self, node: int): # and check does we have node in heap
        return node in self.map

In [4]:
from collections import defaultdict
import heapq

class Graph:
    def __init__(self):
        self.nodes_num = 0
        self.edges_num = 0
        self.E = defaultdict(list)
        self.W = {}
    
    def read_file(self, file_name: str):
        with open(file_name) as f:
            lines = [line.split('\n')[0] for line in f.readlines()]
            self.nodes_num, self.edges_num = list(map(int, lines[0].split(' ')))
            for line in lines[1:]:
                if line != '':
                    node1, node2, weight = list(map(int, line.split(' ')))
                    self.W[(node1, node2)] = self.W[(node2, node1)] = weight
                    self.E[node1].append(node2)
                    self.E[node2].append(node1)
                
    def prims_stupid(self) -> int:
        X = set([1])
        weight_sum = 0
        while len(X) < self.nodes_num:
            min_weight = float('inf')
            for edge in self.W:
                if (edge[0] in X) != (edge[1] in X):
                    if self.W[edge] < min_weight:
                        min_weight = self.W[edge]
                        min_edge = edge
            X.add(min_edge[1])
            X.add(min_edge[0])
            weight_sum += min_weight
        return weight_sum
    
    def prims_heap(self) -> int:
        self.X = set([1])
        V_X = MinHeapMap()
        for node in range(2, self.nodes_num + 1):
            dist = self.W.get((1, node), float('inf'))
            V_X.insert((dist, node))
        weight_sum = 0
        while V_X:
            min_weight, min_node = V_X.pop_min()
            weight_sum += min_weight
            self.X.add(min_node)
            for node2 in self.E[min_node]:
                if node2 in V_X:
                    old_dist, _node2 = V_X.pop_node(node2)
                    new_dist = min(old_dist, self.W[(min_node, node2)])
                    V_X.insert((new_dist, node2))
        return weight_sum

G = Graph()
G.read_file('edges.txt')

-3612829
-3612829


In [8]:
%%timeit
G.prims_stupid()

201 ms ± 17.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit
G.prims_heap()

10.6 ms ± 165 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
